In [279]:
import requests
import re
from urllib.request import urlretrieve
import time
import pytesseract
from PIL import Image
import parsel
import os
import datetime
import xlrd
import xlwt
from xlutils.copy import copy
import pandas as pd
import numpy as np
import openpyxl

In [283]:
FILE1 = '爬虫-排号和排产.xlsx'

In [102]:
def PP_spider():
    timestamp = int(round(time.time() * 1000))

    url1 = 'https://news.oilchem.net/20/0713/15/7f70fc04ccd2542d.html'
    url2 = 'https://passport.oilchem.net/member/login/getImgCode?timestamp='+str(timestamp)
    url3 = 'https://passport.oilchem.net/member/login/checkImgCode?code={code}'
    url4 = 'https://passport.oilchem.net/member/login/login'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    }

    requests.get(url1, headers=headers)
    requests.get(url2, headers=headers)
    urlretrieve(url2, './PP_code.png')
    image = Image.open('./PP_code.png')
    content = pytesseract.image_to_string(image)
    requests.get(url3.format(code=content), headers=headers)
    response = requests.post(url4, {'username': 'rockyeah', 'password': 'dc483e80a7a0bd9ef71d8cf973673924', 'target': 'https://news.oilchem.net/20/0713/15/7f70fc04ccd2542d.html', 'errorPaw': 'a123456'})
    return response.text

In [246]:
def parse_en(enterprise, content):
    pattern = re.compile(r"[-\dA-Za-z]+", re.S)
    res = []
    for c in content:
        temp = re.findall(pattern, c)
        if len(temp) == 0:
            res.append(["停车:"+c])
        else:
            res.append(temp)
    return dict(zip(enterprise, res))

In [284]:
def to_excel(text):
    obj = parsel.Selector(text)
    title = obj.xpath("//div[@id='content']//tr[@class='firstRow']/td/text()").getall()
    temp = obj.xpath("//div[@id='content']//tr[not(contains(@class, 'firstRow'))]").getall()
    content = []
    for t in temp:
        content.append(parsel.Selector(t).xpath("//td/text()").extract())
    file_name = 'PP_' + str(datetime.date.today()) + '.xls'
    flag = os.path.exists(file_name)
    if not flag:
        with open(file_name, 'w', encoding='utf-8') as f:
            pass
    workbook = xlwt.Workbook(encoding='ascii')
    table = workbook.add_sheet('Sheet1')
    for i in range(len(title)):
        table.write(0, i, title[i])
    row_index = 1
    for c in content:
        if len(c) == 5:
            for i in range(len(c)):
                table.write(row_index, i, c[i])
        elif len(c) == 4:
            for i in range(len(c)):
                table.write(row_index, i+1, c[i])
        else:
            for i in range(len(c)):
                table.write(row_index, i+2, c[i])
        row_index += 1
    workbook.save(file_name)
    df = pd.read_excel(file_name, sheet_name='Sheet1', index_col=False)
    part1 = df.iloc[:, 1]
    insert1 = []
    temp_name = None
    temp_index = 0
    for p in part1:
        if pd.isnull(p):
            insert1 = insert1[:-1]
            insert1.append(temp_name+str(temp_index)+'#')
            temp_index += 1
            insert1.append(temp_name+str(temp_index)+'#')
        else:
            temp_name = p
            temp_index = 1
            insert1.append(p)
    df.insert(2, '石化名称备注', insert1)
#     workbook = xlwt.Workbook(encoding='ascii')
#     table = workbook.add_sheet('Sheet1')
#     for t in range(len(df.columns.values)):
#         table.write(0, t, df.columns.values[t])
#     temp_value = None
#     for i in range(len(df.columns.values)):
#         temp_index = 0
#         for j in range(len(df.iloc[:, i])):
#             if not pd.isnull(df.iloc[:, i][j]):
#                 if j != 0:
#                     table.write_merge(temp_index+1, j, i, i, temp_value)
#                 temp_index = j
#                 temp_value = str(df.iloc[:, i][j]) if isinstance(df.iloc[:, i][j], np.int64) else df.iloc[:, i][j]
#         table.write_merge(temp_index+1, len(df.iloc[:, i]), i, i, temp_value)
#     workbook.save(file_name)
    aim_file = pd.read_excel(FILE1, sheet_name='PP排号', index_col=False)
    num = [n for n in aim_file['型号']]
    type_name = [t for t in aim_file['类别']]
    num_type = dict(zip(num, type_name))
    
    equip_dyn = parse_en(insert1, df['装置动态'])
    enterprise_type = {}
    for key in equip_dyn:
        flag = 0
        for t in equip_dyn[key]:
            if t.startswith("停车"):
                enterprise_type[key] = "停车"
                flag = -1
                continue
            type_name = num_type.get(t, None)
            if type_name:
                flag = 1
                enterprise_type[key] = type_name
                continue
        if flag == 0:
            enterprise_type[key] = "找不到对应类型"
    today = datetime.date.today().strftime("%Y/%m/%d")
    origin_excel = xlrd.open_workbook(FILE1)
    sheet = origin_excel.sheet_by_name('PP排产')
    cols = sheet.ncols
    excel = copy(origin_excel)
    table = excel.get_sheet(-1)
    table.write(3, 0, today)
    for i in range(1, cols):
        table.write(3, i, enterprise_type.get(sheet.cell(0, i).value))
    excel.save(FILE1)

In [285]:
if __name__ == '__main__':
    text = PP_spider()
    to_excel(text)